<a href="https://colab.research.google.com/github/ranieri-unimi/faces-malchiodi-2022/blob/main/faces.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### run once

In [ ]:
!pip install pyspark
!pip install findspark

In [ ]:
import os
os.environ["KAGGLE_USERNAME"] = 'ranieriunimi'
os.environ["KAGGLE_KEY"] = str(hex(232307088475198570779809482024044346960))[2:]

In [ ]:
ref = 'bwandowando/ukraine-russian-crisis-twitter-dataset-1-2-m-rows'
!mkdir datasets
!kaggle datasets download $ref --unzip -p ./datasets

## ukraine

In [ ]:
SAMPLE_SIZE = 10*1000

In [ ]:
import numpy as np
import pandas as pd
import csv
import re
import string
import random

In [ ]:
import pyspark
import findspark

In [ ]:
# load dataset 
filename = r"./datasets/UkraineCombinedTweetsDeduped20220227-131611.csv.gzip"
pd.set_option("display.max_columns", None)
df = pd.read_csv(filename, compression='gzip', index_col=0, encoding='utf-8', quoting=csv.QUOTE_ALL)

## data cleaning

In [ ]:
df = df[df.language == 'en']

In [ ]:
df = df.text.tolist()

In [ ]:
df = random.sample(df, SAMPLE_SIZE) if SAMPLE_SIZE else df

In [ ]:
def preprocess_tweet_text(tweet):
    tweet = tweet.lower()
    tweet = re.sub(r"http\S+|www\S+|https\S+", '', tweet, flags=re.MULTILINE)
    tweet = tweet.translate(str.maketrans(string.punctuation+'…’”“‘', ' '*(len(string.punctuation)+5)))  # puntctuation to spaces
    tweet = ' '.join(tweet.strip().split())
    return tweet

## hadoop instance

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext

In [ ]:
rdd = sc.parallelize(enumerate(df))

In [ ]:
rdd.take(7)

In [ ]:
def shiftkv(x) : return (x[-1], x[0])

In [ ]:
dataPipe = (rdd
            .mapValues(preprocess_tweet_text)
            .map(shiftkv)
            .reduceByKey(min)
            .map(shiftkv)
)

In [ ]:
dataPipe.take(7)

In [ ]:
dataPipe.count()

### foos

In [ ]:
!pip install crc16
import crc16
import binascii

### shingles

In [ ]:
SH_LEN = 9

In [ ]:
shigles = dataPipe.flatMap(lambda x : [(x[0], x[-1][i:i+SH_LEN]) for i in range(len(x[-1])-SH_LEN)]).distinct()

In [ ]:
shigles.take(7)

In [ ]:
def hashval(x):
  bval = bytes(x, 'utf-8')
  hval = binascii.crc32(bval) # 4 bytes
  return hval

In [ ]:
charactMx = shigles.mapValues(hashval).distinct()

In [ ]:
charactMx.take(7)

### MinHashing

In [ ]:
def RN(stop, start = 1) : return random.randrange(start,stop)
def RB(B = 4) : return RN(2**(8*B-1),0)*2+1
def PF(x, scale, shift, base) : return (scale * x + shift) % base

In [ ]:
import random
n_permfoo, b_bands = 15, 3

(1 / b_bands)**(1 / (n_permfoo//b_bands)) # threshold

In [ ]:
params = [ {'shift':RB(), 'scale':RB(), 'base':2**(8*4)} for _ in range(n_permfoo) ]
params

In [ ]:
def gen_perm(x):
  doc, shingle = x
  return [ ((doc, h), (shingle, h) ) for h in range(n_permfoo)]

In [ ]:
minHashSignMx = (charactMx
                 .flatMap(gen_perm)
                 .reduceByKey(lambda a, b : a if PF(a[0], **params[a[-1]]) < PF(b[0], **params[a[-1]]) else b)
                 .mapValues(lambda v : v[0])
)
# (doc, hash) , shingle

In [ ]:
minHashSignMx.take(7)

### LSH

In [ ]:
scale, shift = RN(b_bands), RN(b_bands)
bandmap = (lambda x : PF(x, scale, shift, b_bands))

In [ ]:
def band_expand(x):
  (doc, h), v = x
  return ((doc, bandmap(h)), (h, v))

def band_reduct(x):
  (doc, band), a = x
  return ((hash(tuple(a)), band), doc)

In [ ]:
bandMx = (minHashSignMx              # (doc, hashperm) , valshin
          .map(band_expand)             # (doc, band) , (hashperm , valshin)
          .groupByKey().mapValues(list)    # (doc, band) , [ (hashperm , valshin), (hashperm , valshin) ,(hashperm , valshin), ...]
          .mapValues(lambda a : sorted(a, key=lambda x: x[0]))   # (doc, band) , [ (hashperm , valshin), (hashperm , valshin) ,(hashperm , valshin), ...] but sorted
          .mapValues(lambda a : [v for _, v in a] )      # (doc, band) , [valshin, valshin, valshin]
)

In [ ]:
bandMx.take(7)

In [ ]:
bandBuckets = (bandMx
    .map(band_reduct)                           # (HHH_valshin, band), doc
    .groupByKey().mapValues(list)           # (HHH_valshin, band), [doc, doc, doc]
)

In [ ]:
lst = bandBuckets.filter(lambda x : len(x[1])>1).collect()

In [ ]:
lst

In [ ]:
for v in dict(lst).values():
  for i in v:
    print(df[i])
    print('---------------')
  print('___________________________________________________________________________________________')